# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Feature Selection

**Grupo 30: Datatouille**

**http://fdelmazo.github.io/7506-Datos/**

# AL SER UN PROCESO MUY COSTOSO EN TIEMPO Y MEMORIA, SE GUARDAN LOS RESULTADOS FINALES EN VARIABLES

In [ ]:
def get_feature_selection():
    # Se esconde en un def para poder importarlo

    return {
        'features_con_saltos_progresivo':['total_session_conversions',
                                         'days_to_last_conversion',
                                         'doy_last_event',
                                         'total_conversions_month_3',
                                         'has_conversion_month_2'],

        'best_features_progresivo': ['timestamp_last_event',
                                     'total_events',
                                     'avg_events_per_session',
                                     'timestamp_last_checkout',
                                     'total_sessions',
                                     'total_session_conversions',
                                     'days_to_last_checkout',
                                     'total_checkouts_month_5',
                                     'doy_last_checkout',
                                     'woy_last_checkout',
                                     'total_checkouts',
                                     'days_to_last_conversion',
                                     'dom_last_event',
                                     'dom_last_checkout',
                                     'days_to_last_event',
                                     'doy_last_event',
                                     'days_to_last_viewed_product',
                                     'dow_last_event',
                                     'timestamp_last_conversion',
                                     'has_checkout_month_5',
                                     'dow_last_checkout',
                                     'total_session_checkouts',
                                     'total_checkouts_last_week',
                                     'total_checkouts_months_1_to_4',
                                     'woy_last_event',
                                     'doy_last_conversion',
                                     'dow_last_conversion',
                                     'dom_last_conversion'],
        
        'best_features_forward': ['dow_last_conversion', 'has_conversion_last_week', 
                                 'total_conversions_month_4', 'total_session_checkouts', 
                                 'doy_last_conversion', 'timestamp_last_event', 
                                 'dow_last_checkout', 'total_checkouts', 
                                 'has_checkout', 'doy_last_checkout', 
                                 'has_checkout_month_1', 'timestamp_last_checkout', 
                                 'total_sessions', 'woy_last_event', 'has_checkout_month_5', 
                                 'avg_events_per_session'],
        
        'features_con_saltos_forward': ['woy_last_checkout',
                                         'has_conversion_month_5',
                                         'has_checkout_months_1_to_4',
                                         'has_conversion_month_2',
                                         'dom_last_viewed_product'],
        
        'best_features_backward' : ['total_checkouts', 'total_conversions', 'total_events', 
                                    'total_sessions', 'total_session_checkouts', 'total_session_conversions', 
                                    'avg_events_per_session', 'has_checkout', 'total_conversions_month_1', 
                                    'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 
                                    'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 
                                    'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3',
                                    'has_checkout_month_3', 'has_conversion_month_3', 'has_checkout_month_4',
                                    'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5',
                                    'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4',
                                    'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 
                                    'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 
                                    'has_conversion_last_week', 'amount_of_months_that_has_bought', 'timestamp_last_event', 
                                    'timestamp_last_checkout', 'timestamp_last_conversion', 'timestamp_last_viewed_product',
                                    'days_to_last_event', 'days_to_last_checkout', 'days_to_last_viewed_product', 'doy_last_event', 
                                    'dow_last_event', 'dom_last_event', 'woy_last_event', 'doy_last_checkout', 'dow_last_checkout', 
                                    'dom_last_checkout', 'woy_last_checkout', 'doy_last_conversion', 'dow_last_conversion', 
                                    'dom_last_conversion', 'doy_last_viewed_product', 'dow_last_viewed_product',
                                    'dom_last_viewed_product', 'woy_last_viewed_product'] 

}

---
---
---


In [ ]:
import nbimporter # pip install nbimporter
import pandas as pd
from parameter_tuning import get_hiper_params
import submission_framework as SF

df_users = pd.read_csv('data/user-features.csv',low_memory=False).set_index('person')
df_y = pd.read_csv('data/labels_training_set.csv').groupby('person').sum()

seed=42
hiper_params = get_hiper_params()

df_users.head()

Usando Random Forest, el algoritmo más estable de los definidos (XGBoost es poco estable) encontramos que combinación de features es la mas favorable (con la métrica Area Under Curve).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_name = 'random_forest'
params = hiper_params[model_name]
model = RandomForestClassifier(**params,random_state=seed)
random_forest = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,random_forest)

## Cumulative Importance

Se parte de una lista de todos los features ordeandos segun importancia, y se genera una lista de listas acumulativa de esto. Es decir de `[a,b,c]` se pasa a `[ [a], [a,b], [a,b,c] ]`

Esto se hace porque se esta buscando el 'codo': Los features que hacen que incremente el AUC.

In [ ]:
feature_importances = SF.get_feature_importances(df_users,df_y,random_forest)
features_ordenados = feature_importances.index.tolist()
lista_progresiva_de_cols = [features_ordenados[:i] for i in range(1,len(features_ordenados))]

model, max_auc = SF.full_framework_wrapper(df_users,df_y,random_forest)
best_features_progresivo = features_ordenados
features_con_saltos_progresivo = []

for i, cols in enumerate(lista_progresiva_de_cols):
    print(f'\n\nIteración {i+1} de {len(lista_progresiva_de_cols)}\n\n')
    model, auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=cols,verbosity=1)
    if auc > max_auc + 0.0001:
        max_auc = auc
        best_features_progresivo = cols
        features_con_saltos_progresivo.append(cols[-1])
        
best_features_progresivo

Model: random_forest - Accuracy: 0.9496 - AUC: 0.7578 - AUCPR:0.1491


Iteración 1 de 63


Model: random_forest - Accuracy: 0.9496 - AUC: 0.7659 - AUCPR:0.1427
['timestamp_last_checkout']


Iteración 2 de 63


Model: random_forest - Accuracy: 0.9496 - AUC: 0.7634 - AUCPR:0.1379
['timestamp_last_checkout', 'woy_last_checkout']


Iteración 3 de 63


Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCPR:0.1435
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout']


Iteración 4 de 63


Model: random_forest - Accuracy: 0.9496 - AUC: 0.7653 - AUCPR:0.1377
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout', 'dom_last_checkout']


Iteración 5 de 63


Model: random_forest - Accuracy: 0.9496 - AUC: 0.7645 - AUCPR:0.1359
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout', 'dom_last_checkout', 'total_checkouts_month_5']


Iteración 6 de 63


Model: random_forest - Accuracy: 0.9496 - AUC: 0.7654 - AUCPR:0.1377
['timestamp_last_checkout', '

Model: random_forest - Accuracy: 0.9496 - AUC: 0.7906 - AUCPR:0.1471
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout', 'dom_last_checkout', 'total_checkouts_month_5', 'has_checkout_month_5', 'days_to_last_checkout', 'total_session_conversions', 'dow_last_checkout', 'has_checkout_last_week', 'total_sessions', 'timestamp_last_event', 'total_checkouts_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'days_to_last_event', 'timestamp_last_conversion', 'amount_of_months_that_has_bought', 'has_conversion_last_week', 'days_to_last_conversion', 'total_checkouts', 'doy_last_event', 'dow_last_event']


Iteración 24 de 63


Model: random_forest - Accuracy: 0.9496 - AUC: 0.7603 - AUCPR:0.1441
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout', 'dom_last_checkout', 'total_checkouts_month_5', 'has_checkout_month_5', 'days_to_last_checkout', 'total_session_conversions', 'dow_last_checkout', 'has_checkout_last_week', 'total_sessions', 'ti

Model: random_forest - Accuracy: 0.9496 - AUC: 0.8141 - AUCPR:0.1522
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout', 'dom_last_checkout', 'total_checkouts_month_5', 'has_checkout_month_5', 'days_to_last_checkout', 'total_session_conversions', 'dow_last_checkout', 'has_checkout_last_week', 'total_sessions', 'timestamp_last_event', 'total_checkouts_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'days_to_last_event', 'timestamp_last_conversion', 'amount_of_months_that_has_bought', 'has_conversion_last_week', 'days_to_last_conversion', 'total_checkouts', 'doy_last_event', 'dow_last_event', 'dom_last_event', 'doy_last_conversion', 'dow_last_conversion', 'dom_last_conversion', 'woy_last_conversion', 'doy_last_viewed_product', 'dow_last_viewed_product', 'dom_last_viewed_product', 'woy_last_viewed_product', 'last_conversion_sku', 'woy_last_event']


Iteración 35 de 63


Model: random_forest - Accuracy: 0.9496 - AUC: 0.7823 - AUCPR:0.1479
['tim

Model: random_forest - Accuracy: 0.9496 - AUC: 0.7883 - AUCPR:0.1433
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout', 'dom_last_checkout', 'total_checkouts_month_5', 'has_checkout_month_5', 'days_to_last_checkout', 'total_session_conversions', 'dow_last_checkout', 'has_checkout_last_week', 'total_sessions', 'timestamp_last_event', 'total_checkouts_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'days_to_last_event', 'timestamp_last_conversion', 'amount_of_months_that_has_bought', 'has_conversion_last_week', 'days_to_last_conversion', 'total_checkouts', 'doy_last_event', 'dow_last_event', 'dom_last_event', 'doy_last_conversion', 'dow_last_conversion', 'dom_last_conversion', 'woy_last_conversion', 'doy_last_viewed_product', 'dow_last_viewed_product', 'dom_last_viewed_product', 'woy_last_viewed_product', 'last_conversion_sku', 'woy_last_event', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'has_conversion_month_2', 'total_e

Model: random_forest - Accuracy: 0.9496 - AUC: 0.7791 - AUCPR:0.1409
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout', 'dom_last_checkout', 'total_checkouts_month_5', 'has_checkout_month_5', 'days_to_last_checkout', 'total_session_conversions', 'dow_last_checkout', 'has_checkout_last_week', 'total_sessions', 'timestamp_last_event', 'total_checkouts_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'days_to_last_event', 'timestamp_last_conversion', 'amount_of_months_that_has_bought', 'has_conversion_last_week', 'days_to_last_conversion', 'total_checkouts', 'doy_last_event', 'dow_last_event', 'dom_last_event', 'doy_last_conversion', 'dow_last_conversion', 'dom_last_conversion', 'woy_last_conversion', 'doy_last_viewed_product', 'dow_last_viewed_product', 'dom_last_viewed_product', 'woy_last_viewed_product', 'last_conversion_sku', 'woy_last_event', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'has_conversion_month_2', 'total_e

Model: random_forest - Accuracy: 0.9496 - AUC: 0.7581 - AUCPR:0.1404
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout', 'dom_last_checkout', 'total_checkouts_month_5', 'has_checkout_month_5', 'days_to_last_checkout', 'total_session_conversions', 'dow_last_checkout', 'has_checkout_last_week', 'total_sessions', 'timestamp_last_event', 'total_checkouts_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'days_to_last_event', 'timestamp_last_conversion', 'amount_of_months_that_has_bought', 'has_conversion_last_week', 'days_to_last_conversion', 'total_checkouts', 'doy_last_event', 'dow_last_event', 'dom_last_event', 'doy_last_conversion', 'dow_last_conversion', 'dom_last_conversion', 'woy_last_conversion', 'doy_last_viewed_product', 'dow_last_viewed_product', 'dom_last_viewed_product', 'woy_last_viewed_product', 'last_conversion_sku', 'woy_last_event', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'has_conversion_month_2', 'total_e

Model: random_forest - Accuracy: 0.9496 - AUC: 0.7486 - AUCPR:0.1390
['timestamp_last_checkout', 'woy_last_checkout', 'doy_last_checkout', 'dom_last_checkout', 'total_checkouts_month_5', 'has_checkout_month_5', 'days_to_last_checkout', 'total_session_conversions', 'dow_last_checkout', 'has_checkout_last_week', 'total_sessions', 'timestamp_last_event', 'total_checkouts_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'days_to_last_event', 'timestamp_last_conversion', 'amount_of_months_that_has_bought', 'has_conversion_last_week', 'days_to_last_conversion', 'total_checkouts', 'doy_last_event', 'dow_last_event', 'dom_last_event', 'doy_last_conversion', 'dow_last_conversion', 'dom_last_conversion', 'woy_last_conversion', 'doy_last_viewed_product', 'dow_last_viewed_product', 'dom_last_viewed_product', 'woy_last_viewed_product', 'last_conversion_sku', 'woy_last_event', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'has_conversion_month_2', 'total_e

['timestamp_last_checkout',
 'woy_last_checkout',
 'doy_last_checkout',
 'dom_last_checkout',
 'total_checkouts_month_5',
 'has_checkout_month_5',
 'days_to_last_checkout',
 'total_session_conversions',
 'dow_last_checkout',
 'has_checkout_last_week',
 'total_sessions',
 'timestamp_last_event',
 'total_checkouts_last_week',
 'days_to_last_viewed_product',
 'timestamp_last_viewed_product',
 'days_to_last_event',
 'timestamp_last_conversion',
 'amount_of_months_that_has_bought',
 'has_conversion_last_week',
 'days_to_last_conversion',
 'total_checkouts',
 'doy_last_event',
 'dow_last_event',
 'dom_last_event',
 'doy_last_conversion',
 'dow_last_conversion',
 'dom_last_conversion',
 'woy_last_conversion',
 'doy_last_viewed_product',
 'dow_last_viewed_product',
 'dom_last_viewed_product',
 'woy_last_viewed_product',
 'last_conversion_sku',
 'woy_last_event']

##  Forward Selection

Se parte de una lista vacía y se van agregando todos los features uno por uno.

In [ ]:
features = df_users.columns.tolist()

cantidad_features = len(features)

max_auc = 0
features_no_usadas = features
features_ya_usadas = []
best_features_forward = []
features_con_saltos_forward = []

for i in range(cantidad_features+1):
    print(f'\n\nIteración {i} de {cantidad_features}')
    print(f'Lo mejor al momento: {best_features_forward} (AUC: {max_auc:.4f}) \n\n')
    max_local_auc = 0
    for f in features_no_usadas:
        features_a_usar = features_ya_usadas + [f]
        model, auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features_a_usar,verbosity=1)
        if auc > max_local_auc + 0.0001:
            max_local_auc = auc
            feature_a_agregar = f
        
    features_ya_usadas += [feature_a_agregar]
    if features_no_usadas: features_no_usadas.remove(feature_a_agregar)
    
    if max_local_auc > max_auc + 0.0001:
        max_auc = max_local_auc
        best_features_forward = features_ya_usadas[:]
        features_con_saltos_forward.append(feature_a_agregar)
        
best_features_forward



Iteración 0 de 62
Lo mejor al momento: [] (AUC: 0.0000) 


Model: random_forest - Accuracy: 0.9496 - AUC: 0.6438 - AUCPR:0.0731
['total_checkouts']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.5000 - AUCPR:0.0504
['total_conversions']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.6169 - AUCPR:0.0741
['total_events']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.6449 - AUCPR:0.0799
['total_sessions']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.5000 - AUCPR:0.0504
['total_session_checkouts']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.6815 - AUCPR:0.1096
['total_session_conversions']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.5570 - AUCPR:0.0572
['avg_events_per_session']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.6854 - AUCPR:0.1092
['has_checkout']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.5000 - AUCPR:0.0504
['has_conversion']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.5000 - AUCPR:0.0504
['total_conversions_month_1']
Model: random_forest

Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCPR:0.1435
['doy_last_checkout', 'has_checkout_month_3']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCPR:0.1435
['doy_last_checkout', 'has_conversion_month_3']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCPR:0.1435
['doy_last_checkout', 'total_conversions_month_4']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCPR:0.1435
['doy_last_checkout', 'total_checkouts_month_4']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCPR:0.1435
['doy_last_checkout', 'has_checkout_month_4']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCPR:0.1435
['doy_last_checkout', 'has_conversion_month_4']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCPR:0.1435
['doy_last_checkout', 'total_conversions_month_5']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCPR:0.1435
['doy_last_checkout', 'total_checkouts_month_5']
Model: random_forest - Accuracy: 0.9496 - AUC: 0.7684 - AUCP

KeyboardInterrupt: 

##  Backward Elimination

Se parte de una lista con todos los features y se van sacando uno por uno, en busqueda de cual hace que incremente un AUC una vez que se lo remueva.

In [ ]:
features = df_users.columns.tolist()
cantidad_features = len(features)

model, max_auc = SF.full_framework_wrapper(df_users,df_y,random_forest,verbosity=1)
max_local_auc = max_auc
best_features_backward = features
features_ya_usadas = features
features_con_saltos_backward = []

for i in range(cantidad_features+1):
    print(f'\n\nIteración {i} de {cantidad_features}')
    print(f'Lo mejor al momento: {best_features_backward} (AUC: {max_auc:.4f}) \n\n')
    for f in features_ya_usadas:
        features_a_usar = features_ya_usadas[:]
        features_a_usar.remove(f)
        model, auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features_a_usar,verbosity=1)
        if auc > max_local_auc + 0.0001:
            max_local_auc = auc
            feature_a_borrar = f
            
    if feature_a_borrar in features_ya_usadas: features_ya_usadas.remove(feature_a_borrar)
        
    if max_local_auc > max_auc + 0.0001:
        max_auc = max_local_auc
        best_features_backward = features_ya_usadas[:]
        features_con_saltos_backward.append(feature_a_borrar)
        
        
# Ojo, features_con_saltos_backward se refiere a aquellos features que al removerlos se ve un incremento. 
# Es decir, son los features que menos sirven, correr el algoritmo solo sobre esto tendría resultados muy pobres.
best_features_backward